In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [985 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-21 01:50:44--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.35MB/s    in 0.2s    

2022-10-21 01:50:44 (5.35 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName(" Bias_of_Vine_Reviews ").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [ ]:
cleaned_df = df.dropna()
cleaned_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = cleaned_df.select(["review_id","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|                Y|
|R15V54KBMTQWAY|          4|            0|          0|   N|                N|
| RY8I449HNXSVF|          5|            0|          0|   N|                Y|
|R18TLJYCKJFLSR|          5|            0|          0|   N|                Y|
|R1NK26SWS53B8Q|          5|            0|          0|   N|                Y|
|R11LOHEDYJALTN|          5|            0|          0|   N|                Y|
|R3ALQVQB2P9LA7|          4|            0|          0|   N|                Y|
|R3MWLXLNO21PDQ|          5|            0|          0|   N|                Y|
|R2L15IS24CX0LI|          5|            0|          0|   N|                Y|
|R1DJ8976WPWVZU|          3|            0|          0|   N|     

In [ ]:
# Step 1: Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
from pyspark.sql.functions import col
valuable_votes_df = vine_df.filter(col("total_votes") >= 20)
valuable_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
|R1GU6IYZQWQE8X|          2|           24|         25|   N|     

In [ ]:
# Step 2: Retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
percent_helpful_votes_df = valuable_votes_df.withColumn('percent_votes',col('helpful_votes')/col('total_votes')).alias('percent_votes').filter(col("percent_votes") >= 0.5)
percent_helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|0.9152542372881356|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|0.7142857142857143|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|0.9523809523809523|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|0.6363636363636364|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|0.9574468085106383|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|0.9423076923076923|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|0.6666666666666666|
|R2EE2TR4MRDV0U|          5|  

In [ ]:
# Step 3: Retrieves all the rows where a review was written as part of the Vine program (paid)
vine_program_df = percent_helpful_votes_df.filter(col("vine") == "Y")
vine_program_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1MAOLI5FJHAFM|          4|          249|        261|   Y|                N|0.9540229885057471|
| R9PYAUDIBJVEC|          4|           12|         22|   Y|                N|0.5454545454545454|
| R6V9SHMMG5M8F|          5|          101|        110|   Y|                N|0.9181818181818182|
|R37PVLT6ELL5J4|          4|          181|        209|   Y|                N|0.8660287081339713|
| R2FSFGWZF24V9|          4|           50|         51|   Y|                N|0.9803921568627451|
|R3SRW1E8J56IGV|          5|          262|        281|   Y|                N|0.9323843416370107|
| R86Z11D4CWOFM|          4|           32|         36|   Y|                N|0.8888888888888888|
| RNP01HW9YISJO|          4|  

In [ ]:
# Step 4: Retrieves all the rows where the review was not part of the Vine program (unpaid)
out_of_program_df = percent_helpful_votes_df.filter(col("vine") == "N")
out_of_program_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|0.9152542372881356|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|0.7142857142857143|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|0.9523809523809523|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|0.6363636363636364|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|0.9574468085106383|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|0.9423076923076923|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|0.6666666666666666|
|R2EE2TR4MRDV0U|          5|  

In [15]:
# Step 5: Determine the total number of reviews, the number of 5-star reviews
all_reviews = valuable_votes_df.count()
all_5_star_reviews = valuable_votes_df[vine_df["star_rating"] == 5].count()


In [16]:
print(f"======================================================================")
print(f"ALL reviews = {all_reviews}")
print(f"all reviews with 5 star rating = {all_5_star_reviews}")

ALL reviews = 72553
all reviews with 5 star rating = 31600


In [ ]:
# The percentage of 5-star reviews for the two types of review (paid vs unpaid).
from pyspark.sql.functions import col,when,count,lit
ratings_total_df = percent_helpful_votes_df.groupBy("vine").agg(
    count(col("vine")).alias("Total_Reviews"),
    count(when(col("star_rating") == 5, True)).alias("Total_5Star"),
    (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("percentage_of_5Star")).show()

+----+-------------+-----------+-------------------+
|vine|Total_Reviews|Total_5Star|percentage_of_5Star|
+----+-------------+-----------+-------------------+
|   Y|          613|        222| 36.215334420880914|
|   N|        64934|      30530|  47.01697107832569|
+----+-------------+-----------+-------------------+

